In [ ]:
from ngsolve import *
import netgen.gui
from netgen.geom2d import *

import matplotlib.pyplot as plt
import numpy as np
import scipy.sparse as sp

In [ ]:
geo = SplineGeometry()
a = 3
cringx = a/2
cringy = a/3
rring = 0.8
geo.AddRectangle((0,0), (a,a), bc = "outer", leftdomain=1, rightdomain=0)
z = 2
#geo.AddRectangle((0,0), (a,a), bc=['bottom','right','top','left'], leftdomain=1, rightdomain=0)
p1,p2,p3,p4 = [ geo.AppendPoint(x,y) for x,y in [(z,z), (1/2+z,z), (1/2+z,1+z), (z,1+z)] ]
print(p1)

#geo.Append (["line", p1, p2], bc = "hals", leftdomain=3, rightdomain=2)
#geo.Append (["line", p2, p3], bc = "hals", leftdomain=3, rightdomain=2)
#geo.Append (["line", p3, p4], bc = "hals", leftdomain=3, rightdomain=2)
#geo.Append (["line", p4, p1], bc = "hals", leftdomain=3, rightdomain=2)

geo.AddCircle(c=(cringx,cringy), r=rring, bc = "inner", leftdomain=2, rightdomain=1)

geo.SetMaterial (1, "outer")
geo.SetMaterial (2, "inner")

#geo.Append (["spline3", p2, p3, p4])
#geo.Append (["line", p4, p1])

mesh = Mesh(geo.GenerateMesh(maxh=0.1))
Draw(mesh)

In [ ]:
maxind = 20
fes = H1(mesh, order=5,complex = True)
#fes = H1(mesh, order=5,dirichlet=".*")
gfu = GridFunction(fes,name = "u")
erg = np.zeros([len(gfu.vec),maxind],dtype = complex)
fh = np.zeros(len(gfu.vec),dtype = complex)

piha = 1

source = exp(-50**2*((x-a/2)*(x-a/2)+(y-2.5)*(y-2.5)))
Draw(source,mesh,name = "pulse")

In [ ]:
u = fes.TrialFunction()
v = fes.TestFunction()

A = BilinearForm(fes)
K = BilinearForm(fes)
M = BilinearForm(fes)
F = LinearForm(fes)
R = BilinearForm(fes)

omega = Parameter(1)
A += (grad(u)*grad(v)- omega**2*u*v)*dx
K += grad(u)*grad(v)*dx
M += u*v*dx
R += -1j*u*v*ds
A += -1j*omega*u*v*ds # R


#A += 1j*omega*u*v*ds # R 
F += source*v*dx  
o = []  

norm = []

#o = verg[0:21]
for i in range(0,maxind):
    o.append(i)

j = 0
Draw(gfu)
for i in o:   
    omega.Set(i) 
    A.Assemble()
    K.Assemble()
    M.Assemble()
    F.Assemble()  
    R.Assemble() 
    gfu.vec.data = A.mat.Inverse(fes.FreeDofs()) * F.vec
    erg[:,j] = gfu.vec
    fh = F.vec
    norm.append(sqrt (Integrate ((gfu)*(gfu),mesh)))
    print("Number of iter: ",j,"omega: ",i,"L2-Norm:",sqrt (Integrate ((gfu)*(gfu),mesh)))
    j = j+1
#As = A.mat
Ms = M.mat
Ks = K.mat
Rs = R.mat
rows,cols,vals = Ms.COO()
Mh = sp.csr_matrix((vals,(rows,cols)))
rows,cols,vals = Ks.COO()
Kh = sp.csr_matrix((vals,(rows,cols)))
rows,cols,vals = Rs.COO()
Rh = sp.csr_matrix((vals,(rows,cols)))
#rows,cols,vals = As.COO()
#Ah = sp.csr_matrix((vals,(rows,cols)))
#Ah = sp.csr_matrix.toarray(Ah)

In [ ]:
plt.plot(o[1:20],norm[1:20])
plt.xlabel("omega")
plt.ylabel("Euklidic-Norm")
#plt.savefig('results/resonanz_20.png')

In [ ]:
V, r = np.linalg.qr(erg)
V = erg.dot(np.linalg.inv(r))
print(np.vdot(V[:,1],(V[:,2])))
len(V)

In [ ]:
MN = np.transpose(V).dot(Mh.dot(V))
KN = np.transpose(V).dot(Kh.dot(V))
fN = np.transpose(V).dot(fh)
RN = np.transpose(V).dot(Rh.dot(V))

In [ ]:
iteration = int(100)
interval = 20
factor = interval / iteration
omegaN = []
norm_L2 =[]
uN = np.zeros([maxind,iteration])
for i in range(iteration):
    omegaN.append(factor*i)
    uN[:,i] = np.linalg.solve(KN-omegaN[i]**2*MN+omegaN[i]*RN, fN)
    new = uN[:,i].dot(np.transpose(V))
    gif = GridFunction(fes,name = "gif")
    gif.vec.FV().NumPy()[:] = new
    Draw(gif)
    print("i: ",i,"Omega: ",omegaN[i])
    input()
    Redraw()
    #norm_L2.append(sqrt(uN[:,i].dot(uN[:,i])))

In [ ]:



iteration = int(1e4)
interval = 20
factor = interval / iteration
omegaN = []
norm_L2 =[]
res = []
uN = np.zeros([maxind,iteration])
for i in range(iteration):
    print(i)
    omegaN.append(factor*i)
    uN[:,i] = np.linalg.solve(KN-omegaN[i]**2*MN, fN)
    res.append(np.linalg.norm((Kh-omegaN[i]**2*Mh).dot(uN[:,i].dot(np.transpose(V)))-fN.dot(np.transpose(V))))
    norm_L2.append(sqrt(uN[:,i].dot(uN[:,i])))

In [ ]:


#plt.semilogy(verg[0:maxpoints],point[0:maxpoints],'*r')
plt.semilogy(omegaN[10:],norm_L2[10:],'b')
plt.xlabel("omega")
plt.ylabel("Euclidic Norm")
#plt.savefig('results/resonanz_RB_20.png')

In [ ]:
plt.semilogy(omegaN,res)
plt.xlabel("omega")
plt.ylabel("Euclidic Norm")